In [1]:
import pandas as pd
import streamlit as st
import duckdb
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pmdarima import auto_arima
from meteostat import Point, Hourly
import holidays
import warnings
warnings.filterwarnings("ignore")
def load_data():
    con = duckdb.connect(database=':memory:', read_only=False)
    path=r"C:\Users\Shaaf\Desktop\Data Science\Practice Projects\Transport Planning\Sampled_Data\combined_sampled_data.parquet"
    df = con.execute(f"SELECT * FROM '{path}'").df()
    return df
data = load_data()


In [2]:
data=data[data['total_amount']>0]
data=data[data['trip_distance']<=100]
data['Date']=data['tpep_pickup_datetime'].dt.date
data['Hour']=data['tpep_pickup_datetime'].dt.hour
data['Date']=pd.to_datetime(data['Date'])
data=data[(data['Date'].dt.year)==2025]

In [3]:
data['passenger_count'].fillna(0, inplace=True)
data['RatecodeID'].fillna(-1, inplace=True)
data['store_and_fwd_flag'].fillna('N', inplace=True)
data['congestion_surcharge'].fillna(0, inplace=True)
data['Airport_fee'].fillna(0, inplace=True)

In [5]:
data.dtypes

VendorID                          int32
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int32
DOLocationID                      int32
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
Airport_fee                     float64
cbd_congestion_fee              float64
Date                     datetime64[ns]
Hour                              int32
dtype: object

In [6]:
df= data.copy()

In [8]:
df = df.sort_values(['VendorID', 'tpep_pickup_datetime'])

In [9]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,Date,Hour
239220,1,2025-01-01 00:00:00,2025-01-01 01:03:09,1.0,6.40,1.0,Y,211,164,1,...,0.5,17.05,0.0,1.0,74.05,2.5,0.0,0.00,2025-01-01,0
261959,1,2025-01-01 00:00:31,2025-01-01 00:08:31,1.0,1.00,1.0,N,234,137,2,...,0.5,0.00,0.0,1.0,14.30,2.5,0.0,0.00,2025-01-01,0
330710,1,2025-01-01 00:01:28,2025-01-01 00:16:56,4.0,3.60,1.0,N,233,41,1,...,0.5,5.70,0.0,1.0,28.40,2.5,0.0,0.00,2025-01-01,0
248129,1,2025-01-01 00:02:14,2025-01-01 00:10:58,4.0,1.20,1.0,N,79,170,1,...,0.5,0.00,0.0,1.0,15.00,2.5,0.0,0.00,2025-01-01,0
299682,1,2025-01-01 00:02:29,2025-01-01 00:05:08,1.0,0.50,1.0,N,188,188,2,...,0.5,0.00,0.0,1.0,7.60,0.0,0.0,0.00,2025-01-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644442,7,2025-10-31 23:35:42,2025-10-31 23:35:42,1.0,1.13,1.0,N,79,144,1,...,0.5,3.57,0.0,1.0,21.42,2.5,0.0,0.75,2025-10-31,23
3665299,7,2025-10-31 23:47:45,2025-10-31 23:47:45,1.0,3.82,1.0,N,107,142,1,...,0.5,3.00,0.0,1.0,33.45,2.5,0.0,0.75,2025-10-31,23
3897956,7,2025-10-31 23:52:24,2025-10-31 23:52:24,1.0,1.76,1.0,N,163,263,1,...,0.5,3.43,0.0,1.0,20.58,2.5,0.0,0.75,2025-10-31,23
3955391,7,2025-10-31 23:52:34,2025-10-31 23:52:34,4.0,1.03,1.0,N,144,4,1,...,0.5,3.85,0.0,1.0,23.10,2.5,0.0,0.75,2025-10-31,23


In [11]:
# 2. Shift the drop-off info to the next row to compare Trip A and Trip B
df['prev_dropoff_time'] = df.groupby('VendorID')['tpep_dropoff_datetime'].shift(1)
df['prev_dropoff_zone'] = df.groupby('VendorID')['DOLocationID'].shift(1)

In [12]:
#3. Calculate "Idle Time" (Time between drop-off and next pick-up)
df['idle_time_mins'] = (df['tpep_pickup_datetime'] - df['prev_dropoff_time']).dt.total_seconds() / 60

In [16]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,Date,Hour,prev_dropoff_time,prev_dropoff_zone,idle_time_mins
239220,1,2025-01-01 00:00:00,2025-01-01 01:03:09,1.0,6.40,1.0,Y,211,164,1,...,1.0,74.05,2.5,0.0,0.00,2025-01-01,0,NaT,NaN,NaN
261959,1,2025-01-01 00:00:31,2025-01-01 00:08:31,1.0,1.00,1.0,N,234,137,2,...,1.0,14.30,2.5,0.0,0.00,2025-01-01,0,2025-01-01 01:03:09,164.0,-62.633333
330710,1,2025-01-01 00:01:28,2025-01-01 00:16:56,4.0,3.60,1.0,N,233,41,1,...,1.0,28.40,2.5,0.0,0.00,2025-01-01,0,2025-01-01 00:08:31,137.0,-7.050000
248129,1,2025-01-01 00:02:14,2025-01-01 00:10:58,4.0,1.20,1.0,N,79,170,1,...,1.0,15.00,2.5,0.0,0.00,2025-01-01,0,2025-01-01 00:16:56,41.0,-14.700000
299682,1,2025-01-01 00:02:29,2025-01-01 00:05:08,1.0,0.50,1.0,N,188,188,2,...,1.0,7.60,0.0,0.0,0.00,2025-01-01,0,2025-01-01 00:10:58,170.0,-8.483333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644442,7,2025-10-31 23:35:42,2025-10-31 23:35:42,1.0,1.13,1.0,N,79,144,1,...,1.0,21.42,2.5,0.0,0.75,2025-10-31,23,2025-10-31 23:34:04,144.0,1.633333
3665299,7,2025-10-31 23:47:45,2025-10-31 23:47:45,1.0,3.82,1.0,N,107,142,1,...,1.0,33.45,2.5,0.0,0.75,2025-10-31,23,2025-10-31 23:35:42,144.0,12.050000
3897956,7,2025-10-31 23:52:24,2025-10-31 23:52:24,1.0,1.76,1.0,N,163,263,1,...,1.0,20.58,2.5,0.0,0.75,2025-10-31,23,2025-10-31 23:47:45,142.0,4.650000
3955391,7,2025-10-31 23:52:34,2025-10-31 23:52:34,4.0,1.03,1.0,N,144,4,1,...,1.0,23.10,2.5,0.0,0.75,2025-10-31,23,2025-10-31 23:52:24,263.0,0.166667


In [17]:
# If idle_time is high and the zone changed, the driver had to 'cruise' for a fare.
inefficient_trips = df[df['idle_time_mins'] > 15]

In [18]:
inefficient_trips

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,Date,Hour,prev_dropoff_time,prev_dropoff_zone,idle_time_mins
70440,1,2025-01-02 01:41:45,2025-01-02 02:15:36,2.0,18.50,2.0,N,132,43,2,...,1.0,82.69,2.5,1.75,0.00,2025-01-02,1,2025-01-02 01:06:15,68.0,35.500000
104799,1,2025-01-02 02:21:34,2025-01-02 02:26:33,1.0,1.40,1.0,N,229,164,1,...,1.0,13.60,2.5,0.00,0.00,2025-01-02,2,2025-01-02 02:02:43,90.0,18.850000
98170,1,2025-01-02 04:32:24,2025-01-02 04:39:21,1.0,2.70,1.0,N,79,140,1,...,1.0,20.50,2.5,0.00,0.00,2025-01-02,4,2025-01-02 03:39:36,246.0,52.800000
111377,1,2025-01-02 04:56:24,2025-01-02 06:03:40,1.0,5.50,99.0,N,218,80,1,...,0.0,42.00,0.0,0.00,0.00,2025-01-02,4,2025-01-02 04:39:21,140.0,17.050000
167757,1,2025-01-02 12:26:08,2025-01-02 12:46:32,1.0,2.00,1.0,N,43,161,1,...,1.0,26.25,2.5,0.00,0.00,2025-01-02,12,2025-01-02 11:29:58,210.0,56.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3586872,7,2025-10-31 09:48:16,2025-10-31 09:48:16,4.0,0.96,1.0,N,230,237,1,...,1.0,24.42,2.5,0.00,0.75,2025-10-31,9,2025-10-31 09:31:20,186.0,16.933333
3879551,7,2025-10-31 11:37:57,2025-10-31 11:37:57,1.0,1.86,1.0,N,263,74,2,...,1.0,14.00,2.5,0.00,0.00,2025-10-31,11,2025-10-31 11:18:01,238.0,19.933333
3840034,7,2025-10-31 14:32:47,2025-10-31 14:32:47,1.0,1.61,1.0,N,246,230,1,...,1.0,21.90,2.5,0.00,0.75,2025-10-31,14,2025-10-31 14:15:42,237.0,17.083333
3813344,7,2025-10-31 15:08:34,2025-10-31 15:08:34,2.0,1.54,1.0,N,24,239,1,...,1.0,17.64,2.5,0.00,0.00,2025-10-31,15,2025-10-31 14:44:42,162.0,23.866667


In [20]:
(len(inefficient_trips))/len(df)*100

0.16837390724345108

In [22]:
import numpy as np
import pandas as pd

# 1. SETUP: Discretize NYC into Zones (0-262) and Hours (0-23)
n_zones = 263
n_hours = 24
n_actions = 5  # Simplified: 0=Stay, 1=North, 2=South, 3=East, 4=West

# Initialize Q-Table with zeros: [Zone, Hour, Action]
q_table = np.zeros((n_zones, n_hours, n_actions))

# Hyperparameters
alpha = 0.1    # Learning rate
gamma = 0.9    # Discount factor (future rewards matter)
epsilon = 0.1  # Exploration rate

def update_q_table(state, action, reward, next_state):
    zone, hour = state
    next_zone, next_hour = next_state
    
    # Q-Learning Formula: Q(s,a) = Q(s,a) + alpha * [R + gamma * max(Q(s',a')) - Q(s,a)]
    best_next_action = np.argmax(q_table[next_zone, next_hour])
    td_target = reward + gamma * q_table[next_zone, next_hour, best_next_action]
    td_error = td_target - q_table[zone, hour, action]
    
    q_table[zone, hour, action] += alpha * td_error

# 2. TRAINING LOOP (Simulated using NYC Data)
# You would iterate through your trip records here


In [28]:
df['dropoff_hour']=df['tpep_dropoff_datetime'].dt.hour

In [30]:
df['pickup_hour']=df['tpep_pickup_datetime'].dt.hour

In [31]:
# 1. Increase size to cover all IDs up to 265
n_zones = 266  # Now indices 0 through 265 are valid
n_hours = 24
n_actions = 5 

q_table = np.zeros((n_zones, n_hours, n_actions))

# 2. Add a simple clip or filter to your training loop 
# to ensure invalid IDs don't crash the model
for index, trip in df.iterrows():
    # Basic data cleaning: ignore IDs outside the valid range
    if trip['PULocationID'] >= n_zones or trip['DOLocationID'] >= n_zones:
        continue
        
    current_state = (int(trip['PULocationID']), int(trip['pickup_hour']))
    next_state = (int(trip['DOLocationID']), int(trip['dropoff_hour']))
    
    update_q_table(current_state, action, reward, next_state)

In [32]:
q_table

array([[[  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ],
        ...,
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ]],

       [[  4.54715   ,   0.        ,   0.        ,   0.        ,
           0.        ],
        [ 15.60972205,   0.        ,   0.        ,   0.        ,
           0.        ],
        [  8.68314835,   0.        ,   0.        ,   0.        ,
           0.        ],
        ...,
        [ 49.73231436,   0.        ,   0.        ,   0.        ,
           0.        ],
        [ 30.62336491,   0.        ,   0.        ,   0.        ,
           0.    

In [33]:
update_q_table

<function __main__.update_q_table(state, action, reward, next_state)>

In [34]:
import random

def choose_action(state, epsilon):
    zone, hour = state
    # Exploration: pick a random direction/neighbor
    if random.uniform(0, 1) < epsilon:
        return random.randint(0, n_actions - 1)
    # Exploitation: pick the action with the highest Q-value
    else:
        return np.argmax(q_table[zone, hour])

In [35]:
# Constants for NYC
COST_PER_MILE = 0.60  # Fuel and maintenance
OPPORTUNITY_COST_PER_MIN = 0.25 # Value of driver's time

def calculate_reward(trip):
    # Total Revenue
    revenue = trip['fare_amount'] + trip['tip_amount']
    
    # Expenses
    distance_cost = trip['trip_distance'] * COST_PER_MILE
    # If you have idle time data from the previous step:
    # time_cost = trip['idle_time_mins'] * OPPORTUNITY_COST_PER_MIN
    
    return revenue - distance_cost

In [36]:
# Querying the optimized model
my_current_zone = 161 
current_hour = 14

best_action = np.argmax(q_table[my_current_zone, current_hour])
print(f"At 2 PM in Midtown, the optimized action is index: {best_action}")

At 2 PM in Midtown, the optimized action is index: 0
